# Self-Driving Car Engineer Nanodegree


## Project1: **Finding Lane Lines on the Road** 


## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

## Helper Functions

Below helper functions would be used in our lane detection program.

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
            

def calculate_wavg_slope_intercept(lines):
    
    """ This function computes the slope and intercept of all the lines generated by Hough Transform.slope is 
    computed using the formula((y2-y1)/(x2-x1)). Since X=0,Y=0 coordinate is at the top left corner,
    all left lines will have negative slope and all right lines will have positive slopes.
    intercept is calculated using the formula y2-slope*x2.
    
    we use weighted average method to compute left and right lane slopes. In this method, lenghts of lines are multiplied 
    with corresponding slopes to compute the average slope. 
    
    """
        
    # calculate slopes,intercepts and length for all lines. 
    # Categorize the also categorize left and right line
    left_slopes    = []
    left_intercepts = []
    left_lengths  = [] 
    
    right_slopes    = []
    right_intercepts = []
    right_lengths  = [] 
        
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # it is a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y2 - slope*x2
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # It is a left line
                left_slopes.append((slope))
                left_intercepts.append((intercept))
                left_lengths.append((length))
            else:
                right_slopes.append((slope))
                right_intercepts.append((intercept))
                right_lengths.append((length))
    
    # compute weighted average     
    left_wavg_slope  = np.dot(left_lengths,  left_slopes) /np.sum(left_lengths)
    left_wavg_intercept  = np.dot(left_lengths,  left_intercepts) /np.sum(left_lengths)
    right_wavg_slope  = np.dot(right_lengths,  right_slopes) /np.sum(right_lengths)
    right_wavg_intercept  = np.dot(right_lengths,  right_intercepts) /np.sum(right_lengths)
    
    
    return left_wavg_slope,left_wavg_intercept,right_wavg_slope,right_wavg_intercept



def create_lane_lines(image, lines):
    
    # Get the average slope of left and right lane
    left_slope,left_intercept,right_slope,right_intercept = calculate_wavg_slope_intercept(lines)
    
    # Choose the end points.
    y1 = image.shape[0] # bottom of the image
    y2 = int(y1*0.60)   # draw till 2/3 of the screen

    left_x1 = int((y1 - left_intercept)/left_slope)
    left_x2 = int((y2 - left_intercept)/left_slope)
    
    right_x1 = int((y1 - right_intercept)/right_slope)
    right_x2 = int((y2 - right_intercept)/right_slope)
        
    return ((left_x1, y1), (left_x2, y2)), ((right_x1, y1), (right_x2, y2))

    
def draw_lanes(image, lines, color=[255, 0, 0], thickness=20):
    
    # make a copy of the image 
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    
    # Overlay the lanes on actual image.
    # image1 * α + image2 * β + λ
    return cv2.addWeighted(image, 1.0, line_image, 0.95, 0.0)


# Part1:-  Run through all steps of Lane detection algorithm

In [ ]:
# Verify all the steps using 'test_images/solidWhiteRight.jpg' as an example
image = "test_images/solidWhiteRight.jpg"

#read the input image into numpy array
img = mpimg.imread(image)
print('This image is:', type(img), 'with dimensions:', img.shape)
plt.imshow(img)


In [ ]:
#convert into gray scale image. In this algorithm, we are only interested in intesity of pixels
gray = grayscale(img)

# Display the image
plt.imshow(gray, cmap='Greys_r')
  

In [ ]:
# Define a kernel size for Gaussian smoothing / blurring
kernel_size = 5 # Must be an odd number (3, 5, 7...)
blur_gray = gaussian_blur(gray,kernel_size)

# Display the image
plt.imshow(blur_gray, cmap='Greys_r')


In [ ]:
# Define our parameters for Canny and run i
low_threshold = 60
high_threshold = 150
edges = canny(blur_gray, low_threshold, high_threshold)

# Display the image
plt.imshow(edges, cmap='Greys_r')

In [ ]:
# This time we are defining a four sided polygon to mask
imshape = img.shape
vertices = np.array([[(150,imshape[0]),(420,310), (490,310), (imshape[1],imshape[0])]], dtype=np.int32)
masked_img = region_of_interest(edges, vertices)

# Display the image
plt.imshow(masked_img, cmap='Greys_r')

In [ ]:
# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
rho = 2 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 10     # minimum number of votes (intersections in Hough grid cell)
min_line_len = 40 #minimum number of pixels making up a line
max_line_gap = 20    # maximum gap in pixels between connectable line segments

#Call the hough transform routine.
lines = cv2.HoughLinesP(masked_img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)


In [ ]:
#Interopolation algorithm for lane creation
lanes = create_lane_lines(img, lines)
print('This image is:', type(lanes), 'with dimensions:', lanes)

In [ ]:
#plot the lanes on final image
output_img = draw_lanes(img,lanes)

#display the image
plt.imshow(output_img)


# Part2:-  Define the pipeline function and run it through all test images

In [ ]:
def pipeline(img,opath):
    
    gray = grayscale(img)
    
    # Define a kernel size for Gaussian smoothing / blurring
    kernel_size = 5 # Must be an odd number (3, 5, 7...)
    blur_gray = gaussian_blur(gray,kernel_size)

    # Define our parameters for Canny and run it
    low_threshold = 60
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # This time we are defining a four sided polygon to mask
    imshape = img.shape
    vertices = np.array([[(150,imshape[0]),(420,310), (490,310), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_img = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments

    #Call the hough transform routine.
    lines = cv2.HoughLinesP(masked_img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    # identify the left and right lanes using slope and generate lane endpoints using weighted averages
    lanes = create_lane_lines(img, lines)

    # plot the identified gradient lines
    #line_img = np.zeros((masked_img.shape[0], masked_img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)

    #plot the lanes on final image
    output_img = draw_lanes(img,lanes)
    #plt.imshow(output_img)
        
    #copy the result image into output folder
    if opath is not None:
        print ('The output path given is :', opath)
        
        #change the file format 
        outputfile = opath.replace(".jpg",".png")
        plt.imsave(fname=outputfile,arr=output_img)
    
    #return the output image
    return output_img       

**Run the pipeline function on test images first. Once pipeline is confirmed to work, run this routine on test videos ** 

In [ ]:
# iterate all the images through the pipeline routine
for imgfile in os.listdir("test_images/"):
    
    print ('processing image:',imgfile)
    inputfile =  "test_images/"+imgfile
    outputfile = "test_images_output/"+ imgfile    
    pipeline(mpimg.imread(inputfile),outputfile)


#img = mpimg.imread("test_images/solidYellowCurve.jpg")
#pipeline(img,"test_images_output/solidYellowCurve.jpg")



# Part3:-  Run the lane detection function on test video inputs

## Test on Videos
We can test our solution on two provided videos:
`solidWhiteRight.mp4`
`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(img):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image where lines are drawn on lanes)   
   
    result = pipeline(img,None)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))